In [10]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
import re

In [11]:
def prompt_position(position):
    position = input("Enter a job position: ")
    return position


def prompt_location(location):
    location = input("Enter a location (City, State or Zip or remote): ")
    return location


def get_url(position, location):
    url_template = "https://www.indeed.com/jobs?q={}&l={}"
    url = url_template.format(position, location)
    return url


def main(position, location):
    position = prompt_position(position)
    location = prompt_location(location)
    url = get_url(position, location)
    dataframe = pd.DataFrame(columns=["Title", "Company", "Location", "Salary", "Description"])

    # Ensure that the driver path is correct before running this script.

    # Microsoft Windows
    # driver_path = "./drivers/windows/geckodriver.exe"

    # Linux
    driver_path = "/Users/Shreya/Documents/MSDA/DATA-270/PYTHON/chromedriver"

    driver = webdriver.Chrome(executable_path=driver_path)

    postings = 110

    for i in range(0, postings, 10):
        driver.get(url + "&start=" + str(i))
        driver.implicitly_wait(3)

        jobs = driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')
        
        for job in jobs:
            result_html = job.get_attribute('innerHTML')
            soup = BeautifulSoup(result_html, 'html.parser')
            
            tb = soup.find('tbody')
            tr = tb.find('tr')
            td = tr.find('td')
            #print('td \n', td)
            regex = re.compile('jobTitle(.*)')
            t = td.find_all('h2', attrs={'class': regex})
 
            #t =  td.find_all('h2', {'class': 'jobTitle(.*)'})
            #print('\n t \n',t)

            for n in t:

                # Job Title
                job_title = n.find('span').get_text()
                # print("Job Title: " + job_title)

                # Company
                find_company = soup.find('span', class_="companyName")
                if find_company:
                    company = find_company.get_text()
                    # print("Company: " + company)
                else:
                    company = 'None'
                    # print("Company: " + company)

                # Location
                find_job_location = soup.find('div', class_='companyLocation')
                if find_job_location:
                    job_location = find_job_location.get_text()
                    # print("Location: " + job_location)
                else:
                    job_location = 'None'
                    # print("Location: " + job_location)

                # Salary
                find_salary = soup.find('div', class_='metadata salary-snippet-container')
                if find_salary:
                    salary = find_salary.get_text()
                    # print("Salary: " + salary)
                else:
                    salary = 'None'
                    # print("Salary: " + salary)

                # Description
                find_description = soup.find('div', class_='job-snippet')
                if find_description:
                    description = find_description.get_text()
                    # print("Description: " + description)
                else:
                    description = 'None'
                    # print("Description: " + description)

                # print('\n')

                # Add the job posts to the dataframe
                dataframe = dataframe.append({'Title': job_title,
                                              "Company": company,
                                              'Location': job_location,
                                              "Salary": salary,
                                              "Description": description},
                                             ignore_index=True)

    # Convert the dataframe to a csv file
    date = datetime.today().strftime('%Y-%m-%d')
    print(dataframe)
    dataframe.to_csv(date + "_" + position + "_" + location + ".csv", index=False)
    driver.quit()
    


main('', '')

Enter a job position: Engineer
Enter a location (City, State or Zip or remote): San jose


/var/folders/bw/_wkm44713d92hr1xzyckhg1w0000gp/T/ipykernel_70190/526164288.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


                                                 Title  \
0                                  Electrical Engineer   
1                   Mechanical Engineer - Early Career   
2                                        Test Engineer   
3                               Manufacturing Engineer   
4                                     Quality Engineer   
..                                                 ...   
160         Application Engineer (Noise and Vibration)   
161                          CPU Verification Engineer   
162                             Manufacturing Engineer   
163  College Graduate - Industrial Engineering (BS/...   
164                   Associate Manufacturing Engineer   

                                              Company  \
0                                        Confidential   
1                         LOCKHEED MARTIN CORPORATION   
2                                  Proteus Industries   
3                                   Contact Solutions   
4    Streamline El